In [0]:
!wget https://learner.csie.ntu.edu.tw/~judge/ml19spring/ml19spring.zip
!unzip ml19spring.zip

In [66]:
from google.colab import drive
drive.mount('/content/gdrive')

prefix = '/content/gdrive/My Drive/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!rm ml19spring.zip

In [0]:
!unzip /content/gdrive/My\ Drive/colab.zip -d ./

In [0]:
import sys
sys.path.insert(0, './colab/')
from training_utils import *

In [0]:
# specify id
y_id = 1
track_id = 1

In [58]:
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

test_x = np.load('X_test.npz')['arr_0']
train_x = np.load('X_train.npz')['arr_0']
train_y = np.load('Y_train.npz')['arr_0'][:, y_id]
all_x = train_x.copy()
all_y = train_y.copy()
print(test_x.shape, train_x.shape, train_y.shape)

(2500, 10000) (47500, 10000) (47500,)


In [59]:
# pick only important data
idx = {}
with open('colab/29/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1
with open('colab/28/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1

with open('colab/32/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1
with open('colab/35/random_forest' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1
    
idxx = [i[0] for i in idx.items() if i[1] > 1e-3]
print(len(idxx))
train_x = train_x[:, idxx]
test_x = test_x[:, idxx]
all_x = all_x[:, idxx]
print(train_x.shape)

402
(47500, 402)


In [60]:
from sklearn.model_selection import train_test_split

train_x, mytest_x, train_y, mytest_y = train_test_split(train_x, train_y, test_size=0.052631578947368, random_state=1126)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.055555555555554, random_state=1126)
print(train_x.shape, val_x.shape, mytest_x.shape, train_y.shape, val_y.shape, mytest_y.shape)

(42500, 402) (2500, 402) (2500, 402) (42500,) (2500,) (2500,)


In [0]:
# define my own scorer
# actually it should be called error function here
def scorer(y_pred, y_true):
    return 'error', 1.0 * np.sum(np.abs(y_true.get_label() - y_pred)) / len(y_pred)
  
# define my own error function
def mae(y_true, y_pred):
    grad = np.tanh(y_pred - y_true)
    hess = 1 - grad * grad
    return grad, hess

def mae2(y_true, y_pred):
    print(y_true, y_pred)
    grad = (np.exp(10 * (y_pred - y_true)) - 1) / (np.exp(10 * (y_pred - y_true)) + 1)
    hess = (20 * np.exp(10 * (y_pred - y_true))) / (np.exp(10 * (y_pred - y_true)) + 1) ** 2
    return grad, hess

  
params = {
    'objective': mae2,
    'max_depth': 6,
    'learning_rate': 0.1,
    'verbosity': 20,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'n_estimators': 500,
    'n_jobs': 1,
    'subsample': 0.75,
    'booster': 'dart'
}



In [0]:
from xgboost import XGBRegressor
model = XGBRegressor(**params)
file_path = prefix + '48tmp.model'
print(file_path)
model.fit(train_x, train_y, eval_set=[(val_x, val_y)], eval_metric=scorer, early_stopping_rounds=None, xgb_model=file_path)

In [0]:
from xgboost import XGBRegressor
model = XGBRegressor(**params)
model.fit(train_x, train_y, eval_set=[(val_x, val_y)], eval_metric=scorer, early_stopping_rounds=None)
model.save_model('48tmp.model')

In [0]:
model = XGBRegressor(**params)
model.fit(train_x, train_y, eval_set=[(val_x, val_y)], eval_metric=scorer, early_stopping_rounds=None, xgb_model='48tmp.model')

In [55]:
#from xgboost import XGBRegressor
#model = XGBRegressor(**params)
#model.load_model(prefix + '2_46.model')

print("ein1:", err1_calc(model.predict(train_x), train_y, y_id))
print("eval1:", err1_calc(model.predict(val_x), val_y, y_id))
print("etest1:", err1_calc(model.predict(mytest_x), mytest_y, y_id))
print("eall1:", err1_calc(model.predict(all_x), all_y, y_id))

print("ein2:", err2_calc(model.predict(train_x), train_y))
print("eval2:", err2_calc(model.predict(val_x), val_y))
print("etest2:", err2_calc(model.predict(mytest_x), mytest_y))
print("eall2:", err2_calc(model.predict(all_x), all_y))



ein1: 1.2834163040577615
eval1: 1.9987864119140657
etest1: 2.0401483574541928
eall1: 1.3608953651763267
ein2: 0.005913730910476374
eval2: 0.009327857700886016
etest2: 0.009396067317782067
eall2: 0.00627670265772455


In [0]:
# write files
write_prediction(prefix + '48_train_y' + str(y_id) + '_' + str(track_id) + '.txt', 'w', model.predict(all_x).reshape((47500, 1)).astype('str'))
write_prediction(prefix + '48_test_y' + str(y_id) + '_' + str(track_id) + '.txt', 'w', model.predict(test_x).reshape((2500, 1)).astype('str'))
model.save_model(prefix + str(y_id) + '_48.model')

In [0]:
model.save_model(prefix + '48tmp.model')